In [2]:
import pandas as pd
import json

1. Simple Structure

In [ ]:
simple_json_file_path = "simple_structure.json"

2. Multiple Level

In [3]:
multiple_level_json_file_path = "multiple_level.json"

3. Nested Levels

In [4]:
nested_level_json_file_path = "nested_level.json"